<a href="https://colab.research.google.com/github/BillyMuthiani/Ultra_fast-Book_summarizer/blob/main/Book_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
df=pd.read_csv('/content/Goodreads_BestBooksEver_1-10000.csv')
df.head(31)

,url,bookTitle,bookImage,bookAuthors,bookDesc,bookRating,ratingCount,reviewCount,bookPages,bookGenres,bookISBN,recommendations
0,https://www.goodreads.com/book/show/2767052-th...,The Hunger Games,https://i.gr-assets.com/images/S/compressed.ph...,Suzanne Collins,"Could you survive on your own in the wild, wit...",4.32,6717635,176054,374 pages,"Young Adult/31,498|Fiction/17,878|Science Fict...",9.780439e+12,"['Divergent (Divergent, #1)|https://www.goodre..."
1,https://www.goodreads.com/book/show/2.Harry_Po...,Harry Potter and the Order of the Phoenix,https://i.gr-assets.com/images/S/compressed.ph...,"J.K. Rowling,Mary GrandPré",There is a door at the end of a silent corrido...,4.50,2668409,45724,870 pages,"Fantasy/1,797|Young Adult/15,961|Fiction/14,15...",NaN,['Harry Potter and the Cursed Child: Parts One...
2,https://www.goodreads.com/book/show/2657.To_Ki...,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,Harper Lee,The unforgettable novel of a childhood in a sl...,4.28,4772918,95595,324 pages,"Classics/47,203|Fiction/23,575|Historical-Hist...",NaN,['The Great Gatsby|https://www.goodreads.com/b...
3,https://www.goodreads.com/book/show/1885.Pride...,Pride and Prejudice,https://i.gr-assets.com/images/S/compressed.ph...,"Jane Austen,Anna Quindlen",Alternate cover edition of ISBN 9780679783268S...,4.27,3206070,74020,279 pages,"Classics/52,699|Fiction/15,730|Romance/12,874|...",NaN,['Jane Eyre|https://www.goodreads.com/book/sho...
4,https://www.goodreads.com/book/show/41865.Twil...,Twilight,https://i.gr-assets.com/images/S/compressed.ph...,Stephenie Meyer,About three things I was absolutely positive.F...,3.61,5231000,107619,501 pages,"Young Adult/19,982|Fantasy/19,312|Romance/12,0...",9.780316e+12,"['The Hunger Games (The Hunger Games, #1)|http..."
5,https://www.goodreads.com/book/show/19063.The_...,The Book Thief,https://i.gr-assets.com/images/S/compressed.ph...,Markus Zusak,Librarian's note: An alternate cover edition c...,4.38,1954165,117307,552 pages,"Historical-Historical Fiction/20,161|Fiction/1...",9.780376e+12,['All the Light We Cannot See|https://www.good...
6,https://www.goodreads.com/book/show/7613.Anima...,Animal Farm,https://i.gr-assets.com/images/S/compressed.ph...,"George Orwell,Boris Grabnar,Celâl Üster,Celâl ...",George Orwell's timeless and timely allegorica...,3.96,2926888,61574,122 pages,"Classics/34,158|Fiction/19,552|Science Fiction...",9.780452e+12,['The Great Gatsby|https://www.goodreads.com/b...
7,https://www.goodreads.com/book/show/11127.The_...,The Chronicles of Narnia,https://i.gr-assets.com/images/S/compressed.ph...,"C.S. Lewis,Pauline Baynes",Librarian note: An alternate cover for this ed...,4.26,548649,10743,767 pages,"Fantasy/11,823|Classics/4,396|Fiction/3,903|Yo...",9.780066e+12,['J.R.R. Tolkien 4-Book Boxed Set: The Hobbit ...
8,https://www.goodreads.com/book/show/30.J_R_R_T...,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,https://i.gr-assets.com/images/S/compressed.ph...,J.R.R. Tolkien,"This four-volume, boxed set contains J.R.R. To...",4.60,115027,1863,1728 pages,"Fantasy/2,291|Fiction/670|Classics/587|Adventu...",9.780346e+12,['The Chronicles of Narnia (Chronicles of Narn...
9,https://www.goodreads.com/book/show/18405.Gone...,Gone with the Wind,https://i.gr-assets.com/images/S/compressed.ph...,Margaret Mitchell,"Scarlett O'Hara, the beautiful, spoiled daught...",4.30,1106466,20591,1037 pages,"Classics/13,899|Historical-Historical Fiction/...",9.780447e+12,['Scarlett|https://www.goodreads.com/book/show...


In [4]:
#remove duplicated values
df.drop_duplicates(inplace=True)

In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.base import BaseEstimator, TransformerMixin
from difflib import get_close_matches
from rapidfuzz import fuzz
import re

# -------------------------------
# Genre Cleaner
# -------------------------------
def clean_genres(genre_str):
    if pd.isna(genre_str):
        return ""
    genre_str = re.sub(r"[\[\]{}()]", "", genre_str)
    genre_list = re.split(r"[,|/]", genre_str)
    genre_list = [g.strip().split("-")[0].split("/")[0] for g in genre_list if g.strip()]
    genre_list = list(dict.fromkeys(genre_list))  # remove duplicates
    return "|".join(genre_list[:3])  # Keep top 3

# -------------------------------
# Custom Transformers
# -------------------------------
class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.column]

class BookRecommender:
    def __init__(self, n_neighbors=10, fuzzy_threshold=80):
        self.n_neighbors = n_neighbors
        self.fuzzy_threshold = fuzzy_threshold
        self.model = NearestNeighbors(metric='cosine')
        self.feature_pipeline = None
        self.books_df = None
        self.indices = None

    def fit(self, df):
        df = df.copy()
        df['cleanGenres'] = df['bookGenres'].apply(clean_genres)
        df = df.dropna(subset=['bookTitle', 'bookAuthors', 'bookRating'])

        self.books_df = df.reset_index(drop=True)
        self.indices = pd.Series(self.books_df.index, index=self.books_df['bookTitle']).drop_duplicates()

        # Feature pipelines
        title_pipe = Pipeline([
            ('extract', ColumnExtractor('bookTitle')),
            ('tfidf', TfidfVectorizer())
        ])
        author_pipe = Pipeline([
            ('extract', ColumnExtractor('bookAuthors')),
            ('tfidf', TfidfVectorizer())
        ])
        genre_pipe = Pipeline([
            ('extract', ColumnExtractor('cleanGenres')),
            ('tfidf', TfidfVectorizer())
        ])
        numeric_pipe = Pipeline([
            ('extract', ColumnExtractor(['bookRating', 'ratingCount', 'reviewCount'])),
            ('scale', MinMaxScaler())
        ])

        self.feature_pipeline = FeatureUnion([
            ('title', title_pipe),
            ('author', author_pipe),
            ('genres', genre_pipe),
            ('numeric', numeric_pipe)
        ])

        features = self.feature_pipeline.fit_transform(self.books_df)
        self.model.fit(features)
        return self

    def _find_closest_title(self, title):
        all_titles = self.books_df['bookTitle'].tolist()
        matches = get_close_matches(title, all_titles, n=1, cutoff=0)
        if matches:
            score = fuzz.ratio(matches[0].lower(), title.lower())
            return matches[0], score
        return None, 0

    def recommend(self, title, n_recommendations=5, auto_accept_closest=True):
        if title not in self.indices:
            closest, score = self._find_closest_title(title)
            if score >= self.fuzzy_threshold and auto_accept_closest:
                print(f"Using closest match: '{closest}' (score: {score})")
                title = closest
            else:
                raise ValueError(f"Book title '{title}' not found. Closest match '{closest}' (score: {score}).")

        idx = self.indices[title]
        if isinstance(idx, pd.Series):
            idx = idx.iloc[0]
        vec = self.feature_pipeline.transform(self.books_df.iloc[idx:idx + 1])

        distances, indices = self.model.kneighbors(vec, n_neighbors=self.n_neighbors + 10)
        rec_idxs = indices.flatten()

        input_title_clean = title.lower().strip()
        seen_titles = set()
        recommendations = []

        for i in rec_idxs:
            candidate_title = self.books_df.iloc[i]['bookTitle']
            if fuzz.ratio(candidate_title.lower().strip(), input_title_clean) < 90:
                if candidate_title not in seen_titles:
                    seen_titles.add(candidate_title)
                    recommendations.append(i)
            if len(recommendations) >= n_recommendations:
                break

        return self.books_df.iloc[recommendations][['bookTitle', 'bookAuthors', 'bookRating', 'ratingCount', 'reviewCount', 'cleanGenres']].reset_index(drop=True)

# -------------------------------
# Run
# -------------------------------
if __name__ == "__main__":
    df = pd.read_csv('/content/Goodreads_BestBooksEver_1-10000.csv')
    rec = BookRecommender(n_neighbors=10).fit(df)

    query = input("Enter a book title: ")

from tabulate import tabulate

try:
    results = rec.recommend(query, n_recommendations=5)
    print(f"\nTop recommendations for '{query}':")

    table = []
    for i, row in results.iterrows():
        table.append([
            i + 1,
            row['bookTitle'],
            row['bookAuthors'],
            round(row['bookRating'], 2),
            row['cleanGenres']
        ])

    headers = ["#", "Title", "Author(s)", "Rating", "Genres"]
    print(tabulate(table, headers=headers, tablefmt="grid"))

except ValueError as e:
    print(e)


Enter a book title: Romeo and Juliet

Top recommendations for 'Romeo and Juliet':
+-----+-------------------------+---------------------------------------------------------------------+----------+------------------------+
|   # | Title                   | Author(s)                                                           |   Rating | Genres                 |
+=====+=========================+=====================================================================+==========+========================+
|   1 | Measure for Measure     | William Shakespeare,Barbara A. Mowat,Paul Werstine                  |     3.68 | Plays|1|241            |
+-----+-------------------------+---------------------------------------------------------------------+----------+------------------------+
|   2 | Titus Andronicus        | William Shakespeare                                                 |     3.7  | Plays|1|325            |
+-----+-------------------------+---------------------------------------------